In [1]:
from bs4 import BeautifulSoup as bs
import requests
from pprint import pprint

In [2]:
import pandas as pd

In [3]:
import numpy as np

In [4]:
from pymongo import MongoClient

In [5]:
client = MongoClient('127.0.0.1', 27017)

In [6]:
db = client['hw_3']

In [7]:
vacans = db.vacans

In [8]:
url = 'https://spb.hh.ru'
url_add = '/vacancies/data-scientist'
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.93 Safari/537.36'}
response = requests.get(url+url_add, headers=headers)
soup = bs(response.text, 'html.parser')
pager_next = soup.find('a',attrs={'class':'bloko-button', 'data-qa':'pager-next'}).get('href')
job_list = soup.find_all('div',attrs={'class':'vacancy-serp-item'})

In [9]:
try:
    while pager_next:
        for job in job_list:
            job_data = {}
            job_link = job.find('a',attrs={'class':'bloko-link', 'data-qa':'vacancy-serp__vacancy-title'}).get('href')
            job_name = job.find('a',attrs={'class':'bloko-link'}).getText()
            job_salary = job.find('span',attrs={'data-qa': 'vacancy-serp__vacancy-compensation', 'class':'bloko-header-section-3 bloko-header-section-3_lite'})

            if job_salary:
                if job_salary.getText().startswith('от'):
                    job_salary_min = int(job_salary.getText().split(' ')[1].replace('\u202f', ''))
                    job_salary_max = None
                    job_currency = job_salary.getText().split(' ')[2]
                    
                elif job_salary.getText().startswith('до'):
                    job_salary_min = None
                    job_salary_max = int(job_salary.getText().split(' ')[1].replace('\u202f', ''))
                    job_currency = job_salary.getText().split(' ')[2]
                    
                else:
                    job_salary_min = int(job_salary.getText().split(' ')[0].replace('\u202f', ''))
                    job_salary_max = int(job_salary.getText().split(' ')[2].replace('\u202f', ''))
                    job_currency = job_salary.getText().split(' ')[3]
            
            else:
                job_salary_min = None
                job_salary_max = None
                job_currency = None
                
            job_data['link'] = job_link
            job_data['name'] = job_name
            job_data['salary_min'] = job_salary_min
            job_data['salary_max'] = job_salary_max
            job_data['currency'] = job_currency

            vacans.update_one({'link': job_data['link']}, {'$set': job_data}, upsert=True)
            
                
        response = requests.get(url+pager_next, headers=headers)
        soup = bs(response.text, 'html.parser')
        job_list = soup.find_all('div',attrs={'class':'vacancy-serp-item'})
        pager_next = soup.find('a',attrs={'class':'bloko-button', 'data-qa':'pager-next'}).get('href')
        
except AttributeError:
    for job in job_list:
        job_data = {}
        job_link = job.find('a',attrs={'class':'bloko-link', 'data-qa':'vacancy-serp__vacancy-title'}).get('href')
        job_name = job.find('a',attrs={'class':'bloko-link'}).getText()
        job_salary = job.find('span',attrs={'data-qa': 'vacancy-serp__vacancy-compensation', 'class':'bloko-header-section-3 bloko-header-section-3_lite'})

        if job_salary:
            if job_salary.getText().startswith('от'):
                job_salary_min = int(job_salary.getText().split(' ')[1].replace('\u202f', ''))
                job_salary_max = None
                job_currency = job_salary.getText().split(' ')[2]
                
            elif job_salary.getText().startswith('до'):
                job_salary_min = None
                job_salary_max = int(job_salary.getText().split(' ')[1].replace('\u202f', ''))
                job_currency = job_salary.getText().split(' ')[2]
                
            else:
                job_salary_min = int(job_salary.getText().split(' ')[0].replace('\u202f', ''))
                job_salary_max = int(job_salary.getText().split(' ')[2].replace('\u202f', ''))
                job_currency = job_salary.getText().split(' ')[3]
                
                
        else:
            job_salary_min = None
            job_salary_max = None
            job_currency = None
            
        job_data['link'] = job_link
        job_data['name'] = job_name
        job_data['salary_min'] = job_salary_min
        job_data['salary_max'] = job_salary_max
        job_data['currency'] = job_currency

        vacans.update_one({'link': job_data['link']}, {'$set': job_data}, upsert=True)

In [10]:
for i in vacans.find({'$or': [{'salary_min': {'$gt': 100000}},
                              {'salary_max': {'$gt': 100000}}]}):
    pprint(i)

{'_id': ObjectId('610e7749d06e58ce44ed1224'),
 'currency': 'руб.',
 'link': 'https://spb.hh.ru/vacancy/46590676?from=vacancy_search_catalog&query=data%20scientist',
 'name': 'Специалист Computer Vision (Senior Data Scientist)',
 'salary_max': 300000,
 'salary_min': 180000}
{'_id': ObjectId('610e7749d06e58ce44ed1225'),
 'currency': 'руб.',
 'link': 'https://spb.hh.ru/vacancy/46732109?from=vacancy_search_catalog&query=data%20scientist',
 'name': 'Senior Data Scientist',
 'salary_max': 320000,
 'salary_min': 220000}
{'_id': ObjectId('610e7749d06e58ce44ed1229'),
 'currency': 'руб.',
 'link': 'https://spb.hh.ru/vacancy/46497618?from=vacancy_search_catalog&query=data%20scientist',
 'name': 'Senior Data Scientist (в Анапу)',
 'salary_max': 250000,
 'salary_min': 150000}
{'_id': ObjectId('610e7749d06e58ce44ed122c'),
 'currency': 'руб.',
 'link': 'https://spb.hh.ru/vacancy/46743518?from=vacancy_search_catalog&query=data%20scientist',
 'name': 'Data Scientist\\ Аналитик Data Science',
 'salary_m

In [11]:
len(list(vacans.find({})))

114